In [1]:
import numpy as np
import pandas as pd
import numba as nb
from h5py import File
from tqdm import tqdm
import nutils
import common as cm


In [2]:
with File("/mnt/nas/data/1s数据/stk_strN_0_endN_999.h5", "r") as f:
    print(f.keys())
    timestamp = f["datatime"][:]
    midprice = f["mid"][:]
    codes = f["stock_list"][:]

<KeysViewHDF5 ['datatime', 'mid', 'stock_list', 'volume']>


In [3]:
midprice.shape

(10337940, 1000)

In [4]:
timestamp.shape

(10337940, 2)

In [5]:
def convert_nano(data):
    dates = data[:, 0]
    times = data[:, 1]
    date_time = dates * 1e6 + times
    nanoseconds = pd.to_datetime(date_time, format="%Y%m%d%H%M%S").values.astype(
        np.int64
    )
    return nanoseconds


In [6]:
nano_ts = convert_nano(timestamp)

In [7]:
import nutils
from importlib import reload
reload(nutils)
import nutils

In [8]:
from numba import njit, prange
from tqdm import tqdm


def calculate_return(df, timestamp):
    n, m = df.shape
    result = np.zeros((n, m))
    ts = convert_nano(timestamp)
    for i in tqdm(range(m)):
        px = df[:, i]
        ret = nutils.log_return(ts, px, 180) # 180 log return
        result[:, i] = ret
    return result


ret = calculate_return(midprice, timestamp)

100%|██████████| 1000/1000 [25:34<00:00,  1.53s/it]


In [9]:
np.save("../data/n1000_ret.npy",ret)

In [10]:
ret_no_nan = np.where(np.isnan(ret), 0, ret)
ret_no_nan *= 1e4

In [11]:
mkt_index_ret = np.mean(ret_no_nan, axis=1)

In [12]:
# np.save("../data/n1000_mkt_index_ret.npy",mkt_index_ret)

In [13]:
from numba import prange

nb.jit(nopython=True, parallel=True)


def calculate_excess_returns(
    stock_log_returns, stock_index, index_log_returns, index_index, coef
):
    n_stock_timestamps = stock_log_returns.shape[0]
    n_index_timestamps = index_log_returns.shape[0]
    excess_returns = np.full(n_stock_timestamps, np.nan)
    index_pointer = 0
    for i in prange(n_stock_timestamps):
        stock_date, stock_time = stock_index[i]
        while index_pointer < n_index_timestamps and (
            index_index[index_pointer, 0] < stock_date
            or (
                index_index[index_pointer, 0] == stock_date
                and index_index[index_pointer, 1] < stock_time
            )
        ):
            index_pointer += 1
        if index_pointer > 0:
            beta = index_log_returns[index_pointer - 1]
            excess_returns[i] = stock_log_returns[i] - coef * beta

    return excess_returns

In [14]:
import SharedArray as sa
from h5py import File
from tqdm import tqdm
from sklearn.linear_model import LinearRegression


coef_list = []
for cur_code in tqdm(cm.SELECTED_CODES):
    stk_idx = np.where(codes == int(cur_code))[0]
    train_mask = (timestamp[:, 0] < 20210401) & (timestamp[:, 0] >= 20210101)
    train_x = ret_no_nan[train_mask][:, stk_idx]
    train_y = mkt_index_ret[train_mask]
    lr = LinearRegression()
    lr.fit(train_x, train_y)
    coef = lr.coef_
    print(f"{cur_code}:{coef}")
    coef_list.append(coef)

    raw_label = sa.attach(f"label_{cur_code}")
    # raw_timestamp = sa.attach(f"timestamp_{cur_code}")
    with File(f"/mnt/nas/data/股票数据hdf5/stkCode_{cur_code}.h5", "r") as f:
        raw_timestamp = f["timestamp"][:]
    ret = calculate_excess_returns(
        raw_label, raw_timestamp, mkt_index_ret, timestamp, coef
    )
    np.save(
        f"/mnt/disk2/alpha_label/{cur_code}.npy",
        ret.astype(np.float32),
    )

  0%|          | 0/100 [00:00<?, ?it/s]

000537:[0.16026164]


/tmp/ipykernel_4190820/2827726754.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  excess_returns[i] = stock_log_returns[i] - coef * beta
  1%|          | 1/100 [00:26<43:17, 26.23s/it]

000627:[0.13179676]


/tmp/ipykernel_4190820/2827726754.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  excess_returns[i] = stock_log_returns[i] - coef * beta
  2%|▏         | 2/100 [00:45<35:47, 21.91s/it]

...

/tmp/ipykernel_4190820/2827726754.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  excess_returns[i] = stock_log_returns[i] - coef * beta
 98%|█████████▊| 98/100 [30:06<00:32, 16.04s/it]

688777:[0.08473007]


/tmp/ipykernel_4190820/2827726754.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  excess_returns[i] = stock_log_returns[i] - coef * beta
 99%|█████████▉| 99/100 [30:25<00:17, 17.01s/it]

689009:[0.03650756]


/tmp/ipykernel_4190820/2827726754.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  excess_returns[i] = stock_log_returns[i] - coef * beta
100%|██████████| 100/100 [30:47<00:00, 18.48s/it]


In [15]:
codes = codes.flatten()

In [ ]:
import SharedArray as sa

code_ret = sa.attach("label_000537")

In [ ]:
timestamp

In [ ]:
cur_code = "000537"
tmp = np.load(f"/mnt/disk2/alpha_label/{cur_code}.npy")

In [ ]:
tmp